# LLM 과 RAG를 활용한 AI 챗봇 만들기

In [1]:
# 사용환경 준비

import os
from getpass import getpass
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
# os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

In [2]:
# 모델 로드

from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("[2024 한권으로 OK 주식과 세금].pdf")

# 페이지 별 문서 로드
docs = loader.load()

In [3]:
# 문서 청킹

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = text_splitter.split_documents(docs)

# 청킹된 내용을 상위 10개까지 출력
for i, split in enumerate(splits[:10]):
    print(f"Chunk {i+1}:\n{split.page_content}\n")


Chunk 1:
지난해 말 국내 상장법인 주식을 보유한 개인 투자자가 1,400만명을 넘어서는 등  
국민들의 주식시장에 대한 관심이 크게 증가하였습니다.
최근 일반 국민들의 주식투자에 대한 관심이 크게 증가했음에도 불구하고, 주식 투자  
관련 세금문제 등 궁금한 사항에 대하여 도움을 줄 수 있는 안내책자 등은 시중에서  
쉽게 찾아보기 어려운 게 현실입니다.
이에 국세청에서는 주식 관련 각종 세금에 대한 납세자들의 이해를 높이고 납세의무  
이행에 도움이 될 수 있도록 「주식과 세금」 책자를 처음으로 제작·발간하게 되었습니다.
이번에 새롭게 출간하는 ‘주식과 세금’ 책자는 주식거래의 기초상식과 주식의 취득  
부터 보유 및 처분시까지 단계별 세금문제를 총 76개의 문답형식으로 구성하는 한편, 
인포그래픽 등을 적극 활용하여 가독성을 제고하였으며, 구체적인 절세 꿀팁 및 자주 
발생하는 실수 사례 등을 추가하여 활용성도 강화하였습니다.
모쪼록, 이 책자가 주식등 관련 납세자들의 성실한 납세의무 이행에 기여할 수 있기를
기대합니다.
2024.  5
국세청 자산과세국장
머리말

Chunk 2:
본 책자에 수록된 내용은 세법에서 규정하고 있는 내용을 알기 쉽게 요약하여 서술한
것으로 모든 법령규정을 담고 있지는 않습니다.
또한, 법령이 개정되거나 법령의 해석이 변경되어 본 책자의 내용과 다른 경우가 발생
할 수 있으므로 실제 사안에 적용하는 경우 반드시 관련 법령과 해석 등을 충분히 확인
하시기 바랍니다.
본 책자는 발간일 현재 개정된 법령 등을 기준으로 작성되었습니다. 다만, 시행이  
유예된 법령 등은 반영되어 있지 않습니다.
본 책자에 표기된 세율은 지방소득세를 포함하지 않은 세율이며, 법령의 표기방식은  
일부 아래와 같이 간소화하여 표기하였습니다.
본 자료는 주식을 거래할 때 과세되는 세금에 대하여 납세자의 개략적인 이해를 돕기 
위해 일반적인 사항 위주로 수록하였으며, 개별사안에 대해 세법을 적용하는 경우 관련 
법령에 대한 구체적인 검토가 필요합니다.

# CharacterTextSplitter 파라미터 설명

## 기본 개념
텍스트를 작은 청크(chunks)로 나누는 분할기로, 문서를 더 작고 관리하기 쉬운 부분으로 나눕니다.

## 파라미터 상세 설명

### 1. `separator="\n\n"`
- 텍스트를 나눌 구분자 지정
- 여기서는 빈 줄(두 개의 개행문자)을 기준으로 분할
- 문단 단위로 나누기에 적합

### 2. `chunk_size=100`
- 각 청크의 최대 길이
- 여기서는 100자로 제한
- 토큰 처리를 위한 적절한 크기 유지

### 3. `chunk_overlap=10`
- 연속된 청크 간의 중복되는 문자 수
- 여기서는 10자가 중복
- 문맥의 연속성 유지를 위해 사용

### 4. `length_function=len`
- 청크 길이를 측정하는 함수
- `len`: 문자 수를 기준으로 측정
- 다른 함수(예: 토큰 수 계산)로 대체 가능

### 5. `is_separator_regex=False`
- 구분자를 정규표현식으로 처리할지 여부
- `False`: 일반 문자열로 처리
- `True`로 설정 시 정규표현식 패턴 사용 가능


# Chunk Overlap 상세 설명

## 개념
`chunk_overlap`은 연속된 청크들 사이에 중복되는 텍스트의 길이를 지정합니다.

## 작동 방식 예시
원본 텍스트:
"인공지능은 매우 흥미롭습니다. 딥러닝은 인공지능의 한 분야입니다. 머신러닝도 중요한 부분입니다."

### chunk_size=20, chunk_overlap=5 설정 시:

청크1: "인공지능은 매우 흥미롭습"
                    ↓
청크2: "흥미롭습니다. 딥러닝은"
                    ↓
청크3: "러닝은 인공지능의 한"

## 장점
1. 문맥 유지
   - 청크 간 연결성 보장
   - 의미 단절 방지

2. 정보 손실 방지
   - 중요 정보가 청크 경계에 있을 때 보존
   - 키워드 검색 정확도 향상

3. 자연어 처리 성능 향상
   - 문맥 기반 이해도 증가
   - QA 시스템 응답 품질 개선

## 주의사항
- 너무 큰 overlap: 중복 정보 증가, 처리 시간 증가
- 너무 작은 overlap: 문맥 단절 위험

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)

# 청킹된 내용을 상위 10개까지 출력
for i, split in enumerate(splits[:10]):
    print(f"Chunk {i+1}:\n{split.page_content}\n")

Chunk 1:
지난해 말 국내 상장법인 주식을 보유한 개인 투자자가 1,400만명을 넘어서는 등  
국민들의 주식시장에 대한 관심이 크게 증가하였습니다.

Chunk 2:
최근 일반 국민들의 주식투자에 대한 관심이 크게 증가했음에도 불구하고, 주식 투자  
관련 세금문제 등 궁금한 사항에 대하여 도움을 줄 수 있는 안내책자 등은 시중에서

Chunk 3:
쉽게 찾아보기 어려운 게 현실입니다.
이에 국세청에서는 주식 관련 각종 세금에 대한 납세자들의 이해를 높이고 납세의무

Chunk 4:
이행에 도움이 될 수 있도록 「주식과 세금」 책자를 처음으로 제작·발간하게 되었습니다.
이번에 새롭게 출간하는 ‘주식과 세금’ 책자는 주식거래의 기초상식과 주식의 취득

Chunk 5:
부터 보유 및 처분시까지 단계별 세금문제를 총 76개의 문답형식으로 구성하는 한편, 
인포그래픽 등을 적극 활용하여 가독성을 제고하였으며, 구체적인 절세 꿀팁 및 자주

Chunk 6:
발생하는 실수 사례 등을 추가하여 활용성도 강화하였습니다.
모쪼록, 이 책자가 주식등 관련 납세자들의 성실한 납세의무 이행에 기여할 수 있기를
기대합니다.
2024.  5

Chunk 7:
2024.  5
국세청 자산과세국장
머리말

Chunk 8:
본 책자에 수록된 내용은 세법에서 규정하고 있는 내용을 알기 쉽게 요약하여 서술한
것으로 모든 법령규정을 담고 있지는 않습니다.

Chunk 9:
또한, 법령이 개정되거나 법령의 해석이 변경되어 본 책자의 내용과 다른 경우가 발생
할 수 있으므로 실제 사안에 적용하는 경우 반드시 관련 법령과 해석 등을 충분히 확인

Chunk 10:
하시기 바랍니다.
본 책자는 발간일 현재 개정된 법령 등을 기준으로 작성되었습니다. 다만, 시행이  
유예된 법령 등은 반영되어 있지 않습니다.



# RecursiveCharacterTextSplitter 파라미터 설명

## 기본 개념
텍스트를 재귀적으로 더 작은 청크로 분할하는 분할기입니다. 일반 CharacterTextSplitter와 달리 여러 구분자를 순차적으로 시도합니다.

## 파라미터 상세 설명

### 1. `chunk_size=100`
- 각 청크의 최대 길이
- 여기서는 100자로 제한
- 토큰 처리를 위한 적절한 크기 유지

### 2. `chunk_overlap=10`
- 연속된 청크 간의 중복되는 문자 수
- 여기서는 10자가 중복
- 문맥의 연속성 유지를 위해 사용

### 3. `length_function=len`
- 청크 길이를 측정하는 함수
- `len`: 문자 수를 기준으로 측정
- 다른 함수(예: 토큰 수 계산)로 대체 가능

### 4. `is_separator_regex=False`
- 구분자를 정규표현식으로 처리할지 여부
- `False`: 일반 문자열로 처리
- `True`로 설정 시 정규표현식 패턴 사용 가능

## 기본 구분자 우선순위
1. "\n\n" (단락)
2. "\n" (줄바꿈)
3. " " (공백)
4. "" (문자 단위)

## 일반 CharacterTextSplitter와의 차이점
- 여러 구분자를 순차적으로 시도
- 더 자연스러운 텍스트 분할
- 문맥 보존에 더 효과적
- 긴 문서 처리에 더 적합

In [5]:
# 벡터 임베딩 생성 

from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")



In [6]:


# 벡터 스토어 생성

import faiss
from langchain_community.vectorstores import FAISS


vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)


In [7]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText()|   contextual_prompt | model


In [1]:
while True: 
    print("========================")
    query = input("질문을 입력하세요 (종료하려면 'q' 또는 'quit' 입력): ")
    
    if query.lower() in ['q', 'quit']:
        print("프로그램을 종료합니다.")
        break
        
    response = rag_chain_debug.invoke(query)
    print("Final Response:")
    print(response.content)

프로그램을 종료합니다.


# RAG vs 기존 챗봇 비교

## 기존 챗봇의 한계

### 1. 고정된 지식
- 학습된 데이터만 사용 가능
- 최신 정보 반영이 어려움
- 재학습이 필요한 경우 많은 비용과 시간 소요

### 2. 환각(Hallucination) 위험
- 잘못된 정보를 사실처럼 제시할 수 있음
- 답변의 출처를 추적하기 어려움

## RAG의 장점

### 1. 실시간 정보 활용
- 외부 문서나 데이터베이스에서 관련 정보를 검색
- 최신 정보를 즉시 반영 가능
- PDF, 웹페이지 등 다양한 소스 활용 가능

### 2. 정확성 향상
- 검색된 문서를 기반으로 답변 생성
- 답변의 출처 추적 가능
- 환각 현상 감소

### 3. 맞춤형 지식베이스
- 조직이나 목적에 맞는 특정 문서 활용
- 예: 귀하의 코드에서는 "[2024 한권으로 OK 주식과 세금].pdf" 문서를 기반으로 답변

### 4. 비용 효율성
- 전체 모델 재학습 없이 새로운 정보 추가 가능
- 유지보수가 용이